<a href="https://colab.research.google.com/github/SrirajVuppala98/Colab-Notebooks/blob/main/Table_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [85]:
print("Hello World")

Hello World


In [ ]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20250506
    Uninstalling pdfminer.six-20250506:
      Successfully uninstalled pdfminer.six-20250506


In [ ]:
import pdfplumber

with pdfplumber.open("/content/HDFC_Statement-1-3.pdf") as pdf:
    for p in pdf.pages:
        for t in p.extract_tables():
            for r in t:
                print(r)

['Date', 'Narration', 'Chq./Ref.No.', 'Value Dt', 'Withdrawal Amt.', 'Deposit Amt.', 'Closing Balance']
['', '', '', '', '', '', '']
['01/04/24', 'UPI-MRS KUSMA DEWANGAN-PAYTMQR28100505', '0000409274369436', '01/04/24', '10.00', '', '6,531.71']
[None, None, None, None, None, '', None]
['', '', '', '', '', None, '']
['', '010117YG5Q304NLC@PAYTM-YESB0PTMUPI-40927', '', '', '', '', '']
['', None, '', '', '', '', '']
[None, '', None, None, None, None, None]
['', '4369436-PAYMENT FROM PHONE', '', '', '', '', '']
['', None, '', '', '', '', '']
[None, '', None, None, None, None, None]
['01/04/24', 'UPI-TOMAN LAL SAHU-SAGARSAHU97912@IBL-HD', '0000445835370692', '01/04/24', '5,000.00', '', '1,531.71']
[None, None, None, None, None, '', None]
['', '', '', '', '', None, '']
['', 'FC0000788-445835370692-PAYMENT FROM PHON', '', '', '', '', '']
['', None, '', '', '', '', '']
[None, '', None, None, None, None, None]
['', 'E', '', '', '', '', '']
['', None, '', '', '', '', '']
[None, '', None, None, N

In [ ]:
!pip install camelot-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.6/66.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 74.2 MB/s eta 0:00:00


In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 65.8 MB/s eta 0:00:00


In [ ]:
import camelot
import pandas as pd
import fitz
import re

/usr/local/lib/python3.11/dist-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [ ]:
pdf_path = '/content/ICICI-BANK-STATEMENT.pdf'

In [ ]:
def icici_doc_handler(pdf_path):
  transaction_dict = {}
  pdf_document = fitz.open(pdf_path)
  page_count = pdf_document.page_count
  column_name = ['date', 'mode', 'description', 'deposit', 'withdrawal', 'balance']
  transaction_df = pd.DataFrame(columns=column_name)

  for num in range(1, page_count+1):
    cam_tables = camelot.read_pdf(pdf_path, flavor='stream', pages=str(num))

    if len(cam_tables) > 0:
      trans_found = False

      for tables_num in range(len(cam_tables)):
        table = cam_tables[tables_num].df

        for i, row in table.iterrows():
          if str(row[0]).strip().upper() == 'DATE' and str(row[1]).strip().upper() == 'MODE**':
            table.columns = table.iloc[i]          # Set the row as header
            table = table.iloc[i + 1:].reset_index(drop=True)  # Get all rows after the header
            trans_found = True
            break

        if trans_found:
          break

      if trans_found:
        i = 0
        while(i<len(table)):
          if table['PARTICULARS'][i] == 'TOTAL':
            transaction_dict['total_withdrawal'] = table['WITHDRAWALS'][i]
            transaction_dict['total_deposit'] = table['DEPOSITS'][i]
            transaction_dict['final_balance'] = table['BALANCE'][i]
            i += 1
            continue

          j = 0
          counter = 0
          while(table['DATE'][i+counter] == ''):
            counter += 1

          if counter == 0:
            j = 1
          elif counter == 1:
            j = 3
          elif counter == 2:
            j = 5

          date = table['DATE'][i+counter]
          mode = table['MODE**'][i+counter]
          deposits = table['DEPOSITS'][i+counter]
          withdrawals = table['WITHDRAWALS'][i+counter]
          balance = table['BALANCE'][i+counter]

          if i+j < len(table):
            particulars = table['PARTICULARS'].iloc[i:i+j].astype(str).str.cat()
          else:
            particulars = table['PARTICULARS'].iloc[i:].astype(str).str.cat()

          new_row = [date, mode, particulars, deposits, withdrawals, balance]
          transaction_df.loc[len(transaction_df)] = new_row

          i = i+j

  transaction_df['date'] = pd.to_datetime(transaction_df['date'], format='%d-%m-%Y').dt.date
  # transaction_dict['transactions'] = transaction_df.to_dict(orient='records')

  # return transaction_dict
  return transaction_df

In [ ]:
trans_df = icici_doc_handler(pdf_path)
trans_df

,date,mode,description,deposit,withdrawal,balance
0,2019-04-01,,B/F,,,"65,731.31"
1,2019-04-02,DEBIT CARD,VPS/ACT/201904021337/909208028740/HYDERABAD,,"4,377.00","61,354.31"
2,2019-04-03,,IPS REF DT 31 03 2019 032260 HPCL 0 75CASHLESS IN,19.95,,"61,374.26"
3,2019-04-03,,VISA REF SRI SAI BABA FILLING S,15.00,,"61,389.26"
4,2019-04-03,MOBILE BANKING,MMT/IMPS/909318812658/SHINE HAIR/AXIS BANKLTD,"29,600.00",,"90,989.26"
...,...,...,...,...,...,...
701,2020-03-21,,UPI/008114378397/UPI/anand.ganpaa@ok/AxisBank ...,,"1,990.00","2,83,156.85"
702,2020-03-27,DEBIT CARD,VPS/METRO CASH/202003271131/008706262347/HYDER...,,"5,464.00","2,77,692.85"
703,2020-03-30,,020201563824:Int.Pd:31-12-2019 to 29-03-2020,"2,228.00",,"2,79,920.85"
704,2020-03-30,,UPI/009012435096/UPI/vikranthpatel@o/HDFCBANK LTD,,"5,000.00","2,74,920.85"


In [ ]:
trans_df.loc[len(trans_df)] = trans_df.loc[len(trans_df)-1]

In [ ]:
trans_df

,date,mode,description,deposit,withdrawal,balance
0,2019-04-01,,B/F,,,"65,731.31"
1,2019-04-02,DEBIT CARD,VPS/ACT/201904021337/909208028740/HYDERABAD,,"4,377.00","61,354.31"
2,2019-04-03,,IPS REF DT 31 03 2019 032260 HPCL 0 75CASHLESS IN,19.95,,"61,374.26"
3,2019-04-03,,VISA REF SRI SAI BABA FILLING S,15.00,,"61,389.26"
4,2019-04-03,MOBILE BANKING,MMT/IMPS/909318812658/SHINE HAIR/AXIS BANKLTD,"29,600.00",,"90,989.26"
...,...,...,...,...,...,...
703,2020-03-30,,020201563824:Int.Pd:31-12-2019 to 29-03-2020,"2,228.00",,"2,79,920.85"
704,2020-03-30,,UPI/009012435096/UPI/vikranthpatel@o/HDFCBANK LTD,,"5,000.00","2,74,920.85"
705,2020-03-31,DEBIT CARD,VPS/EASY DAY/202003311221/009106519119/HYDERABAD,,196.00,"2,74,724.85"
706,2020-03-31,DEBIT CARD,VPS/EASY DAY/202003311221/009106519119/HYDERABAD,,196.00,"2,74,724.85"


In [ ]:
trans_df.at[706, 'description'] = 'VPS/EASY DAY/202003311332/009106519119/HYDERABAD'
trans_df.at[707, 'description'] = 'VPS/EASY DAY/202023311332/0091065191199/Amr'

In [ ]:
trans_df

,date,mode,description,deposit,withdrawal,balance
0,2019-04-01,,B/F,,,"65,731.31"
1,2019-04-02,DEBIT CARD,VPS/ACT/201904021337/909208028740/HYDERABAD,,"4,377.00","61,354.31"
2,2019-04-03,,IPS REF DT 31 03 2019 032260 HPCL 0 75CASHLESS IN,19.95,,"61,374.26"
3,2019-04-03,,VISA REF SRI SAI BABA FILLING S,15.00,,"61,389.26"
4,2019-04-03,MOBILE BANKING,MMT/IMPS/909318812658/SHINE HAIR/AXIS BANKLTD,"29,600.00",,"90,989.26"
...,...,...,...,...,...,...
703,2020-03-30,,020201563824:Int.Pd:31-12-2019 to 29-03-2020,"2,228.00",,"2,79,920.85"
704,2020-03-30,,UPI/009012435096/UPI/vikranthpatel@o/HDFCBANK LTD,,"5,000.00","2,74,920.85"
705,2020-03-31,DEBIT CARD,VPS/EASY DAY/202003311221/009106519119/HYDERABAD,,196.00,"2,74,724.85"
706,2020-03-31,DEBIT CARD,VPS/EASY DAY/202003311332/009106519119/HYDERABAD,,196.00,"2,74,724.85"


In [ ]:
trans_dict = trans_df.to_dict(orient='records')

In [ ]:
from collections import Counter
from math import sqrt

In [ ]:
# s1 = "VPS/EASY DAY/202003311332/009106519119/Amr"
# s2 = "VPS/EASY DAY/202023311332/009106519119/HYDERABAD"
def cos_sim(s1, s2):
  # Convert strings to character frequency vectors
  vec1 = Counter(s1)
  vec2 = Counter(s2)

  # Calculating cosine similarity
  dot_product = sum(vec1[ch] * vec2[ch] for ch in vec1)
  magnitude1 = sqrt(sum(count ** 2 for count in vec1.values()))
  magnitude2 = sqrt(sum(count ** 2 for count in vec2.values()))
  res = dot_product / (magnitude1 * magnitude2)
  # print(res)
  return res

In [ ]:
import pandas as pd
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.metrics.pairwise import cosine_similarity

def anomaly_detection(trans_df):
  SIMILARITY_THRESHOLD = 0.90
  similar_txns_df = pd.DataFrame()

  for date, group in trans_df.groupby('Date'):
    # Filter withdrawals only
    filtered_group = group[(group['Deposit Amt'] == 0) & (group['Withdrawal Amt.'] != 0)]

    # Skip if fewer than 2 transactions
    if len(filtered_group) < 2:
      continue

    descriptions = filtered_group['Narration'].astype(str).tolist()
    withdrawals = filtered_group['Withdrawal Amt.'].astype(str).tolist()
    indices = filtered_group.index.tolist()

    # Vectorize descriptions
    # vectorizer = TfidfVectorizer().fit_transform(descriptions)
    # similarity_matrix = cosine_similarity(vectorizer)

    matched_indices = set()

    for i in range(len(descriptions)):
      for j in range(i + 1, len(descriptions)):
        similarity = cos_sim(descriptions[i], descriptions[j])
        # Check same withdrawal amount and similarity threshold
        if withdrawals[i] == withdrawals[j] and similarity > SIMILARITY_THRESHOLD:
          idx_i = indices[i]
          idx_j = indices[j]

          if (idx_i, idx_j) not in matched_indices and (idx_j, idx_i) not in matched_indices:
            similar_txns_df = pd.concat([similar_txns_df, trans_df.loc[[idx_i, idx_j]]])
            matched_indices.add((idx_i, idx_j))

  # Final clean-up
  similar_txns_df = similar_txns_df.drop_duplicates().reset_index(drop=True)
  return similar_txns_df

In [ ]:
df = pd.read_csv('/content/df_table.csv')
df

,Unnamed: 0,Date,Narration,Ref No.,Value Dt,Withdrawal Amt.,Deposit Amt,Closing Balance,Net Change
0,0,2025-06-02,CC 000526873XXXXXX3737 AUTOPAY SI-TAD,668434693,02/06/25,866.00,0.0,71045.43,-866.00
1,1,2025-06-03,UPI-BOMMERA GEETANJALI-GEETABOMMERA1010 @OKSBI...,515480531495,04/06/25,0.00,5566.0,76611.43,5566.00
2,2,2025-06-05,UPI-NULI VENKATA RAMA SU-SATISHNULI2003- 2@OKI...,515661690301,05/06/25,0.00,217.0,76828.43,217.00
3,3,2025-06-05,UPI-BOGGARAPU SAI ROHAN-ROHAN.BOGGARAPU@ OKHDF...,105952273959,05/06/25,0.00,217.0,77045.43,217.00
4,4,2025-06-05,UPI-AMRUTHAM CAFETERIA-GPAY-11244456460@ OKBIZ...,105960672577,05/06/25,105.00,0.0,76940.43,-105.00
5,5,2025-06-05,UPI-MR SHANMUKHA RAO CHI-PAYTMQR6BQG0C@P TYS-Y...,105980127592,05/06/25,230.00,0.0,76710.43,-230.00
6,6,2025-06-05,UPI-NUHA SUVIR VARIER-NUHAVARIER@OKICICI -ICIC...,552256742563,05/06/25,0.00,108.0,76818.43,108.00
7,9,2025-06-06,UPI-MR PALINA DANIEL PRA-9948525003@IBL- IDIB0...,495124584477,06/06/25,0.00,280.0,76518.43,280.00
8,8,2025-06-06,UPI-DAALCHINI-PAYTM-7289005034@PTYS-YESB 0PTMU...,106017091835,06/06/25,20.00,0.0,76238.43,-20.00
9,7,2025-06-06,UPI-RAJUGARI LOGILI-PAYTMQR6C9396@PTYS-Y ESB0P...,106016791548,06/06/25,560.00,0.0,76258.43,-560.00


In [ ]:
anomaly_detection(df)

,Unnamed: 0,Date,Narration,Ref No.,Value Dt,Withdrawal Amt.,Deposit Amt,Closing Balance,Net Change
0,39,2025-06-19,UPI-SARAGADA SHIVA AMAR -SHIVAAMARNATH9-,106726118371,19/06/25,100.0,0.0,7350.61,-100.0
1,37,2025-06-19,PHONE UPI-SARAGADA SHIVA AMAR -SHIVAAMARNATH9-,106726090917,19/06/25,100.0,0.0,7550.61,-100.0
2,38,2025-06-19,UPI-SARAGADA SHIVA AMAR -SHIVAAMARNATH9-,106726103647,19/06/25,100.0,0.0,7450.61,-100.0


In [ ]:
descriptions

['VPS/EASY DAY/202003311221/009106519119/HYDERABAD',
 'VPS/EASY DAY/202003311332/009106519119/HYDERABAD',
 'VPS/EASY DAY/202023311332/0091065191199/Amr']

In [ ]:
cos_sim(descriptions[1], descriptions[2])

0.9555821391714722

array([[1.        , 0.58251743, 0.58251743],
       [0.58251743, 1.        , 0.58251743],
       [0.58251743, 0.58251743, 1.        ]])

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def anomaly_detection(trans_dict):
  trans_df = pd.DataFrame(trans_dict)
  trans_df['date'] = pd.to_datetime(trans_df['date'], format='%d-%m-%Y').dt.date

  # Loop through each group by date
  for date, group in trans_df.groupby(trans_df['date']):
      narrations = group['description'].astype(str).tolist()
      indices = group.index.tolist()

      if len(narrations) < 2:
          continue

      # Vectorize narrations
      tfidf = TfidfVectorizer()
      vectors = tfidf.fit_transform(narrations)

      # Compute cosine similarity matrix
      sim_matrix = cosine_similarity(vectors)

      anomaly_df = pd.DataFrame(columns = trans_df.columns)

      dup = []

      # Compare upper triangle of matrix to avoid duplicates
      for i in range(len(narrations)):
          for j in range(i + 1, len(narrations)):
              if sim_matrix[i, j] > 0.85:
                  print(sim_matrix[i, j])
                  # Retrieve full rows from original DataFrame
                  row1 = trans_df.loc[indices[i]].to_dict()
                  row2 = trans_df.loc[indices[j]].to_dict()
                  print(indices[i], indices[j])
                  if((trans_df.loc[indices[i]]['withdrawal'] == trans_df.loc[indices[j]]['withdrawal'])
                  and (trans_df.loc[indices[i]]['withdrawal'] != '')):
                    if indices[i] not in dup:
                      dup.append(indices[i])
                      anomaly_df.loc[len(anomaly_df)] = trans_df.loc[indices[i]]
                    if indices[j] not in dup:
                      dup.append(indices[j])
                      anomaly_df.loc[len(anomaly_df)] = trans_df.loc[indices[j]]

  # anomaly_df = anomaly_df.drop_duplicates()
  # anomaly_dict = anomaly_df.to_dict(orient='records')
  return anomaly_df

In [ ]:
anomaly_df = anomaly_detection(trans_dict)

1.0000000000000002
132 133
1.0000000000000004
258 259
1.0000000000000004
502 503
1.0000000000000004
677 678


In [ ]:
anomaly_df

,date,mode,description,deposit,withdrawal,balance


### PDF Tampering

In [1]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [2]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 37.6 MB/s eta 0:00:00


In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader("/content/CHANDED.pdf")
meta = reader.metadata

print("Created:", meta.get('/CreationDate'))
print("Modified:", meta.get('/ModDate'))

Created: D:20250624110734+05'30'
Modified: D:20250624110734+05'30'


In [ ]:
with open("/content/CHANDED.pdf", "rb") as f:
    content = f.read()
    count = content.count(b'startxref')
    print(f"Number of xref tables: {count}")

Number of xref tables: 2


In [ ]:
import fitz

doc = fitz.open("/content/Sriraj HDFC.pdf")
for page in doc:
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
        if block['type'] == 0:  # text block
            for line in block['lines']:
                for span in line['spans']:
                    print("Font:", span['font'], "Size:", span['size'], "Text:", span['text'])


Font: Times-Bold Size: 8.0 Text: Date
Font: Times-Bold Size: 8.0 Text: Narration
Font: Times-Bold Size: 8.0 Text: Chq./Ref.No.
Font: Times-Bold Size: 8.0 Text: Value Dt
Font: Times-Bold Size: 8.0 Text: Withdrawal Amt.
Font: Times-Bold Size: 8.0 Text: Deposit Amt.
Font: Times-Bold Size: 8.0 Text: Closing Balance
Font: Times-Roman Size: 8.0 Text: 01/04/25
Font: Times-Roman Size: 8.0 Text: UPI-SWIGGY
Font: Times-Roman Size: 8.0 Text: LIMITED-SWIGGY1ONLINE.GPAY@OK
Font: Times-Roman Size: 8.0 Text: 0000102420869515
Font: Times-Roman Size: 8.0 Text: 01/04/25
Font: Times-Roman Size: 8.0 Text: 377.00
Font: Times-Roman Size: 8.0 Text: 56,899.72
Font: Times-Roman Size: 8.0 Text: PAYAXIS-UTIB0000553-102420869515-UPI
Font: Times-Roman Size: 8.0 Text: 01/04/25 UPI-EATCLUB-EATCLUB@PTYBL-YESB0PTMUPI-10
Font: Times-Roman Size: 8.0 Text: 0000102446782812
Font: Times-Roman Size: 8.0 Text: 01/04/25
Font: Times-Roman Size: 8.0 Text: 161.00
Font: Times-Roman Size: 8.0 Text: 56,738.72
Font: Times-Roman Size

In [35]:
import fitz

doc = fitz.open("/content/ICIC_1-3 (2).pdf")
font_size_set = set()

for page in doc:
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
        if block['type'] == 0:  # text block
            for line in block['lines']:
                for span in line['spans']:
                    font_size_pair = (span['font'], round(span['size'], 2))
                    font_size_set.add(font_size_pair)

# Optionally, print them
for font, size in sorted(font_size_set):
    print("Font:", font, "| Size:", size)


Font: Helvetica | Size: 8.0
Font: Helvetica | Size: 9.0
Font: Helvetica | Size: 10.0
Font: RupeeForadian | Size: 8.0
Font: ZurichBT-Roman | Size: 5.0
Font: ZurichBT-Roman | Size: 7.0
Font: ZurichBT-Roman | Size: 8.0
Font: ZurichBT-Roman | Size: 9.0
Font: ZurichBT-Roman | Size: 12.0


Font: Helvetica | Size: 8.0

Font: RupeeForadian | Size: 8.0

Font: ZurichBT-Roman | Size: 5.0

Font: ZurichBT-Roman | Size: 7.0

Font: ZurichBT-Roman | Size: 8.0

Font: ZurichBT-Roman | Size: 9.0

Font: ZurichBT-Roman | Size: 10.0

Font: ZurichBT-Roman | Size: 12.0


In [4]:
# Step 1: Install system dependencies
!apt-get install -y libzbar0

# Step 2: Install required Python packages
!pip install pyzbar pdf2image Pillow

# Step 3: Download poppler (for pdf2image) if needed
!apt-get install -y poppler-utils


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-double3
  libgs9 libgs9-common libidn12 libijs-0.35 libjbig2dec0 libjxr-tools libjxr0
  liblqr-1-0 libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra
  libmagickwand-6.q16-6 libv4l-0 libv4lconvert0 libwmflite-0.2-7 poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre
  ghostscript-x libfftw3-bin libfftw3-dev inkscape poppler-utils
  fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nanum
The following NEW packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts
  imagemagick-6-common libdjvulibre-text libdjvulibre21 libfftw3-

In [5]:
from pdf2image import convert_from_path
from pyzbar.pyzbar import decode

def extract_barcodes_from_first_page(pdf_path):
    images = convert_from_path(pdf_path, dpi=300, first_page=1, last_page=1)
    image = images[0]
    barcodes = decode(image)

    barcode_results = []
    for barcode in barcodes:
        barcode_info = {
            "page": 1,
            "type": barcode.type,
            "data": barcode.data.decode("utf-8"),
            "rect": barcode.rect
        }
        barcode_results.append(barcode_info)

    return barcode_results

# Example usage
pdf_path = "/content/ICICI-BANK-STATEMENT.pdf"  # Upload this manually or via files.upload()
barcodes = extract_barcodes_from_first_page(pdf_path)
for b in barcodes:
    print(f"Page {b['page']}: [{b['type']}] {b['data']}")


Page 1: [CODE128] 020201563824??KASU1008 500018


In [6]:
barcodes[0]['data']

'020201563824??KASU1008 500018'

In [72]:
import fitz  # PyMuPDF
from PyPDF2 import PdfReader
from PIL import Image
import numpy as np


def check_pdf_metadata(pdf_path):
    reader = PdfReader(pdf_path)
    metadata = reader.metadata
    creation_date = metadata.get('/CreationDate')
    mod_date = metadata.get('/ModDate')
    return {
        "creation_date": creation_date,
        "modification_date": mod_date,
        "metadata_mismatch": creation_date != mod_date
    }


def detect_multiple_startxref(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()
        count = content.count(b"startxref")
    print(f"🧭 startxref count: {count}")
    return count

import re

def extract_object_definitions(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()

    object_matches = re.findall(rb"(\d+) (\d+) obj", content)
    object_numbers = {int(obj_num) for obj_num, _ in object_matches}
    print(f"📦 Total unique objects defined: {len(object_numbers)}")
    return sorted(object_numbers)

def extract_xref_references(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()

    xref_blocks = content.split(b"xref")[1:]  # skip any preamble
    xref_object_nums = set()

    for block in xref_blocks:
        lines = block.strip().split(b"\n")
        if not lines:
            continue
        try:
            start, count = map(int, lines[0].split())
            for i in range(count):
                xref_object_nums.add(start + i)
        except:
            continue

    print(f"📌 Object numbers listed in xref: {len(xref_object_nums)}")
    return sorted(xref_object_nums)

def detect_unreferenced_objects(pdf_path):
    defined = set(extract_object_definitions(pdf_path))
    referenced = set(extract_xref_references(pdf_path))
    unreferenced = defined - referenced
    print(f"❗️ Unreferenced objects (orphans): {sorted(unreferenced)}")
    return unreferenced

def analyze_pdf_for_tampering(pdf_path):
    print(f"🔍 Analyzing: {pdf_path}")
    startxref_count = detect_multiple_startxref(pdf_path)
    unreferenced_objects = detect_unreferenced_objects(pdf_path)

    tampered = startxref_count > 1 or len(unreferenced_objects) > 0
    print("🔐 Tampering Detected:" if tampered else "✅ No obvious tampering detected.")
    return tampered

def detect_hidden_or_obfuscated_text(pdf_path):
    doc = fitz.open(pdf_path)
    suspicious_spans = []

    barcodes = extract_barcodes_from_first_page(pdf_path)
    barcode_data = ''
    if barcodes:
      barcode_data = barcodes[0]['data']

    for page_num, page in enumerate(doc, start=1):
        # Render page to image
        pix = page.get_pixmap(matrix=fitz.Matrix(2, 2), alpha=False)
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        img_np = np.array(img)

        blocks = page.get_text("dict")["blocks"]
        for block in blocks:
            if block["type"] == 0:  # text block
                for line in block["lines"]:
                    for span in line["spans"]:
                        bbox = span["bbox"]
                        font_size = round(span.get("size", 0), 2)
                        opacity = span.get("opacity", 1.0)
                        color_int = span.get("color", 0)
                        angle = span.get("rotation", 0)
                        r = (color_int >> 16) & 255
                        g = (color_int >> 8) & 255
                        b = color_int & 255
                        text_rgb = np.array([r, g, b])

                        # Scale bbox for rendered image (matrix=2, so scale by 2)
                        x0, y0, x1, y1 = [int(v * 2) for v in bbox]
                        x0, y0 = max(x0, 0), max(y0, 0)
                        x1, y1 = min(x1, img_np.shape[1]), min(y1, img_np.shape[0])

                        region = img_np[y0:y1, x0:x1]
                        if region.size == 0:
                            continue

                        bg_rgb = np.median(region.reshape(-1, 3), axis=0)  # get median background RGB

                        # Check similarity
                        color_diff = np.linalg.norm(text_rgb - bg_rgb)

                        if (
                            color_diff < 20 or        # very close to background color
                            opacity < 0.1 or
                            font_size < 2 or
                            font_size > 30 or
                            abs(angle) > 0
                        ):
                          if barcode_data != span["text"]:
                            suspicious_spans.append({
                                "page": page_num,
                                "text": span["text"],
                                "font": span["font"],
                                "size": font_size,
                                "opacity": opacity,
                                "rotation": angle,
                                "text_color": text_rgb.tolist(),
                                "bg_color": bg_rgb.tolist(),
                                "color_diff": color_diff
                            })

    return suspicious_spans


# def extract_pdf_object_types(pdf_path):
#     doc = fitz.open(pdf_path)
#     object_types = {}

#     for xref in range(1, doc.xref_length()):
#         try:
#             obj = doc.xref_object(xref)
#             for t in ["/Font"]: #"/Page", , "/Image", "/XObject", "/Annots", "/Metadata", "/Sig"
#                 if t in obj:
#                     object_types[t] = object_types.get(t, 0) + 1
#         except:
#             continue

#     return object_types

def extract_pdf_font_types(pdf_path):
  doc = fitz.open(pdf_path)
  font_size_set = set()

  for page in doc:
    blocks = page.get_text("dict")["blocks"]
    for block in blocks:
      if block['type'] == 0:  # text block
        for line in block['lines']:
          for span in line['spans']:
            font_size_pair = (span['font'], round(span['size'], 2))
            font_size_set.add(font_size_pair)
  return font_size_set


def compare_structures(original_path, suspect_path):
    original = extract_pdf_font_types(original_path)
    suspect = extract_pdf_font_types(suspect_path)

    structural_diff = original != suspect
    return structural_diff

    # all_keys = set(original.keys()).union(set(suspect.keys()))
    # diffs = {}

    # for key in all_keys:
    #     orig_count = original.get(key, 0)
    #     sus_count = suspect.get(key, 0)
    #     if orig_count != sus_count:
    #         diffs[key] = {"original": orig_count, "suspect": sus_count}

    # return diffs

In [73]:
def pdf_integrity_check(original_pdf, suspect_pdf): #
    # 1. Metadata
    meta_result = check_pdf_metadata(suspect_pdf)
    metadata_mismatch = meta_result["metadata_mismatch"]
    print("Creation Date - ", meta_result["creation_date"])
    print("Modification Date - ", meta_result["modification_date"])

    # 2. Xref count
    xref_tampered = analyze_pdf_for_tampering(suspect_pdf)
    # multiple_xrefs = xref_count > 1

    # 3. Hidden/obfuscated text
    hidden_text = detect_hidden_or_obfuscated_text(suspect_pdf)
    print(hidden_text)
    has_hidden_text = len(hidden_text) > 0

    # 4. Structural diff
    has_structure_diff = compare_structures(original_pdf, suspect_pdf)
    # has_structure_diff = len(structure_diffs) > 0

    # Final decision
    tampered = metadata_mismatch or xref_tampered or has_hidden_text or has_structure_diff

    print("Tampered:", tampered)
    return {
        "tampered": tampered,
        "metadata_mismatch": metadata_mismatch,
        "multiple_xrefs": xref_tampered,
        "hidden_text_found": has_hidden_text,
        "structure_differences": has_structure_diff
    }

In [84]:
# Example usage
report = pdf_integrity_check("/content/ICICI-BANK-STATEMENT.pdf", "/content/ICICI-BANK-STATEMENT_Changed.pdf")
# report = pdf_integrity_check("/content/ICICI-BANK-STATEMENT.pdf")

Creation Date -  D:20250625125534+05'30'
Modification Date -  D:20250625125534+05'30'
🔍 Analyzing: /content/ICICI-BANK-STATEMENT_Changed.pdf
🧭 startxref count: 2
📦 Total unique objects defined: 542
📌 Object numbers listed in xref: 7609
❗️ Unreferenced objects (orphans): []
🔐 Tampering Detected:
[]
Tampered: True


In [67]:
report

{'tampered': False,
 'metadata_mismatch': False,
 'multiple_xrefs': False,
 'hidden_text_found': False,
 'structure_differences': False}

In [78]:
import fitz

doc = fitz.open("/content/HDFC_Statement.pdf")
xref_count = doc.xref_length()  # total number of xref objects
print(f"Total xref objects: {xref_count}")


Total xref objects: 21169


In [82]:
xref_id = 3
obj_str = doc.xref_object(xref_id)
print(f"Object {xref_id}:\n{obj_str}")

Object 3:
<<
  /Type /Page
  /Parent 2 0 R
  /Resources <<
    /XObject <<
      /Image5 5 0 R
    >>
    /ExtGState <<
      /GS6 6 0 R
      /GS7 7 0 R
    >>
    /Font <<
      /F1 8 0 R
      /F2 11 0 R
    >>
    /ProcSet [ /PDF /Text /ImageB /ImageC /ImageI ]
  >>
  /Annots [ 10 0 R 13 0 R ]
  /MediaBox [ 0 0 638.04 842.04 ]
  /Contents 4 0 R
  /Group <<
    /Type /Group
    /S /Transparency
    /CS /DeviceRGB
  >>
  /Tabs /S
  /StructParents 0
>>


In [83]:
with open("/content/HDFC_Statement.pdf", "rb+") as f:
  content = f.read()

  # Find last 'startxref' keyword
  pos = content.rfind(b'startxref')
  if pos == -1:
    raise ValueError("startxref not found.")
pos

1569335

In [ ]:
def detect_multiple_startxref(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()
        count = content.count(b"startxref")
    print(f"🧭 startxref count: {count}")
    return count


In [ ]:
import re

def extract_object_definitions(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()

    object_matches = re.findall(rb"(\d+) (\d+) obj", content)
    object_numbers = {int(obj_num) for obj_num, _ in object_matches}
    print(f"📦 Total unique objects defined: {len(object_numbers)}")
    return sorted(object_numbers)


In [ ]:
def extract_xref_references(pdf_path):
    with open(pdf_path, "rb") as f:
        content = f.read()

    xref_blocks = content.split(b"xref")[1:]  # skip any preamble
    xref_object_nums = set()

    for block in xref_blocks:
        lines = block.strip().split(b"\n")
        if not lines:
            continue
        try:
            start, count = map(int, lines[0].split())
            for i in range(count):
                xref_object_nums.add(start + i)
        except:
            continue

    print(f"📌 Object numbers listed in xref: {len(xref_object_nums)}")
    return sorted(xref_object_nums)


In [ ]:
def detect_unreferenced_objects(pdf_path):
    defined = set(extract_object_definitions(pdf_path))
    referenced = set(extract_xref_references(pdf_path))
    unreferenced = defined - referenced
    print(f"❗️ Unreferenced objects (orphans): {sorted(unreferenced)}")
    return unreferenced


In [ ]:
def analyze_pdf_for_tampering(pdf_path):
    print(f"🔍 Analyzing: {pdf_path}")
    startxref_count = detect_multiple_startxref(pdf_path)
    unreferenced_objects = detect_unreferenced_objects(pdf_path)

    tampered = startxref_count > 1 or len(unreferenced_objects) > 0
    print("🔐 Tampering Detected:" if tampered else "✅ No obvious tampering detected.")
    return tampered


In [ ]:
analyze_pdf_for_tampering("/content/ICIC_1-3 (2).pdf")

🔍 Analyzing: /content/ICIC_1-3 (2).pdf
🧭 startxref count: 1
📦 Total unique objects defined: 20
📌 Object numbers listed in xref: 0
❗️ Unreferenced objects (orphans): [1, 2, 11, 12, 13, 14, 15, 16, 17, 19, 21, 31, 32, 33, 34, 35, 36, 37, 45, 46]
🔐 Tampering Detected:


True

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 837.5 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 82.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
!apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
!pip install pymupdf

In [ ]:
from ultralytics import YOLO
import cv2
import pytesseract
from PIL import Image
import re
import numpy as np
import pandas as pd
import json
import fitz  # PyMuPDF
import os
import shutil
from datetime import datetime

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
folder_path = 'processing'
if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
os.mkdir(folder_path)

In [ ]:
# Open the source PDF
pdf_path = "/content/ICICI-BANK-STATEMENT.pdf"
doc = fitz.open(pdf_path)
page_count = doc.page_count

# Select the page number you want to extract (e.g., page 2 -> index 1)
page_number = [0, page_count-1, page_count-2]  # 0-based index

# Create a new PDF with just that one page
for page_num in page_number:
  page = doc.load_page(page_num)

  # Render page to image (as a Pixmap)
  pix = page.get_pixmap(dpi=300)

  # Save the image
  output_image_path = f"{folder_path}/ICICI_page_{page_num + 1}.png"
  pix.save(output_image_path)

# Clean up
doc.close()

In [ ]:
# Load trained model
model = YOLO("/content/best.pt")

img_path = 'processing/ICICI_page_1.png'

# Predict on new image
results = model(img_path)[0]

# Draw bounding boxes
image = cv2.imread(img_path)
for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    conf = float(box.conf[0])
    cls_id = int(box.cls[0])
    label = f"{model.names[cls_id]} {conf:.2f}"
    cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

cv2.imwrite("predicted_image.jpg", image)


image 1/1 /content/processing/ICICI_page_1.png: 640x480 3 Account Detailss, 1 Address, 1 FD Balance, 1 From Date, 1 To Date, 654.5ms
Speed: 15.5ms preprocess, 654.5ms inference, 35.3ms postprocess per image at shape (1, 3, 640, 480)


True

In [ ]:
# Load original image (for OCR)
image = cv2.imread(img_path)

ocr_results = {}
ocr_text = ''

# For each detected bounding box
for box in results.boxes:
  x1, y1, x2, y2 = map(int, box.xyxy[0])
  cls_id = int(box.cls[0])
  label = model.names[cls_id]

  # Crop the region inside the box
  cropped = image[y1:y2, x1:x2]
  cropped_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

  # OCR on cropped region
  text = pytesseract.image_to_string(cropped_pil).strip()  #, config="--psm 6"
  ocr_text += '\n'+text

  # Store result
  if text:
    if label not in ocr_results:
      ocr_results[label] = []
    ocr_results[label].append(text)

In [ ]:
ocr_results

{'Address': ['MR.KASULABADA PRASHANTH\n\n13-1-55/AT/4 PLOT NO 4 AVANTHI NGR THOTA NR\n\nPOCHAMMA TEMPLE MOTHI NGR BALANAGAR SANAT NGR\nREDDY\n\nTELANGANA - INDIA - 500018'],
 'Account Details': ['Summary of Accounts held under Cust ID: 536619964 as on March 31, 2020\nACCOUNT DETAILS - INR',
  'Statement of Transactions in Savings Account Number: 020201563824 in INR for the period',
  'Your Base Branch: 4TH NERELLA HOUSE,\nPANJAGUTTA,,,500034'],
 'FD Balance': ['ACCOUNT TYPE A/c BALANCE(I)f_ FIXED DEPOSITS (LINKED) BAL.(II) TOTAL BALANCE(I+ll) NOMINATION\nSavings A/c 020201563824 2,74,724.85 0.00 2,74,724.85 Registered\nTOTAL 2,74,724.85 0.00 2,74,724.85'],
 'To Date': ['March 31, 2020'],
 'From Date': ['April 01, 2019']}

In [ ]:
for label in ocr_results.keys():
  if(len(ocr_results[label]) > 1):
    for i in range(1, len(ocr_results[label])):
      ocr_results[label][0] += '\n' + ocr_results[label][i]
  ocr_results[label] = ocr_results[label][0]

details = {}

lines = [line.strip() for line in ocr_results['Address'].split('\n') if line.strip()]

details['Name'] = lines[0]
details['Cust_Address'] = ", ".join(lines[1:])
from_date = ocr_results['From Date']

from_date = datetime.strptime(from_date, "%B %d, %Y")
fromatted_from_date = from_date.strftime("%d/%m/%Y")
details['From'] = fromatted_from_date

to_date = ocr_results['To Date']
to_date = datetime.strptime(to_date, "%B %d, %Y")
fromatted_to_date = to_date.strftime("%d/%m/%Y")
details['To'] = fromatted_to_date

lines = [line.strip() for line in ocr_results['Account Details'].split('\n') if line.strip()]

# print(lines)

cust_id = re.search(r'Cust ID:\s*(\d+)', lines[0])

details['Cust ID'] = cust_id.group(1)
currency = lines[1].split('-')[-1].strip()

account_no = re.search(r'Account Number:\s*(\d+)', lines[4])
if account_no != None:
  details['Account No'] = account_no.group(1)

  details['Currency'] = currency
  details['Address'] = lines[2].split(':')[-1].strip() + ' ' + lines[3]

else:
  account_no = re.search(r'Account Number:\s*(\d+)', lines[2])
  details['Account No'] = account_no.group(1)

  details['Currency'] = currency
  details['Address'] = lines[3].split(':')[-1].strip() + ' ' + lines[4]

details['Bank'] = "ICICI"

lines = [line.strip() for line in ocr_results['FD Balance'].split('\n') if line.strip()]
details['Nomination'] = lines[1].split()[-1]

details['FD_Balance'] = lines[2].split()[2]

In [ ]:
details

{'Name': 'MR.KASULABADA PRASHANTH',
 'Cust_Address': '13-1-55/AT/4 PLOT NO 4 AVANTHI NGR THOTA NR, POCHAMMA TEMPLE MOTHI NGR BALANAGAR SANAT NGR, REDDY, TELANGANA - INDIA - 500018',
 'From': '01/04/2019',
 'To': '31/03/2020',
 'Cust ID': '536619964',
 'Account No': '020201563824',
 'Currency': 'INR',
 'Address': '4TH NERELLA HOUSE, PANJAGUTTA,,,500034',
 'Bank': 'ICICI',
 'Nomination': 'Registered',
 'FD_Balance': '0.00'}

In [ ]:
len(results.boxes.cls)

7

In [ ]:
if page_count > 1:
  # Load trained model
  model = YOLO("/content/best.pt")

  img_path = f'processing/ICICI_page_{page_count}.png'

  # Predict on new image
  results = model(img_path)[0]

  # Draw bounding boxes
  image = cv2.imread(img_path)
  for box in results.boxes:
      x1, y1, x2, y2 = map(int, box.xyxy[0])
      conf = float(box.conf[0])
      cls_id = int(box.cls[0])
      label = f"{model.names[cls_id]} {conf:.2f}"
      cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
      cv2.putText(image, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

  cv2.imwrite("predicted_image.jpg", image)


image 1/1 /content/processing/ICICI_page_22.png: 640x480 1 Other Acc Details, 472.6ms
Speed: 5.2ms preprocess, 472.6ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 480)


In [ ]:
type(results.boxes.cls.item())

float

In [ ]:
if len(results.boxes.cls) == 0:
  img_path = f'processing/ICICI_page_{page_count-1}.png'

  # Predict on new image
  results = model(img_path)[0]

In [ ]:
if len(results.boxes.cls) == 1 and results.boxes.cls.item() == 4.0:
  # Load original image (for OCR)
  image = cv2.imread(img_path)

  ocr_results = {}
  ocr_text = ''

  # For each detected bounding box
  for box in results.boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cls_id = int(box.cls[0])
    label = model.names[cls_id]

    # Crop the region inside the box
    cropped = image[y1:y2, x1:x2]
    cropped_pil = Image.fromarray(cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB))

    # OCR on cropped region
    text = pytesseract.image_to_string(cropped_pil).strip()  #, config="--psm 6"
    ocr_text += '\n'+text

    # Store result
    if text:
      if label not in ocr_results:
        ocr_results[label] = []
      ocr_results[label].append(text)

In [ ]:
print(ocr_results['Other Acc Details'])

['Account Related Other Information\n\nRee RENE teers\n\nSavings 020201563824 500229013 ICIC0000202']


In [ ]:
for label in ocr_results.keys():
  if(len(ocr_results[label]) > 1):
    for i in range(1, len(ocr_results[label])):
      ocr_results[label][0] += '\n' + ocr_results[label][i]
  ocr_results[label] = ocr_results[label][0]

In [ ]:
lines = [line.strip() for line in ocr_results['Other Acc Details'].split('\n') if line.strip()]

info = lines[-1].split()

info

['Savings', '020201563824', '500229013', 'ICIC0000202']

In [ ]:
if len(info) < 5:
  details['Nominee_Name'] = ''
else:
  details['Nominee_Name'] = ' '.join(info[4:])

details['Account_type'] = info[0]
details['MICR'] = info[2]
details['IFSC'] = info[3]

details

{'Name': 'MR.KASULABADA PRASHANTH',
 'Cust_Address': '13-1-55/AT/4 PLOT NO 4 AVANTHI NGR THOTA NR, POCHAMMA TEMPLE MOTHI NGR BALANAGAR SANAT NGR, REDDY, TELANGANA - INDIA - 500018',
 'From': '01/04/2019',
 'To': '31/03/2020',
 'Cust ID': '536619964',
 'Account No': '020201563824',
 'Currency': 'INR',
 'Address': '4TH NERELLA HOUSE, PANJAGUTTA,,,500034',
 'Bank': 'ICICI',
 'Nomination': 'Registered',
 'FD_Balance': '0.00',
 'Nominee_Name': '',
 'Account_type': 'Savings',
 'MICR': '500229013',
 'IFSC': 'ICIC0000202'}

In [86]:
!git status

fatal: not a git repository (or any of the parent directories): .git
